# 🎤 Indic Parler TTS - Interactive Audio Quality Control

**Features:**
- 69 Named Speakers
- 21 Indian Languages
- 12 Emotion Tags
- Full Audio Quality Controls

Run the cells below to launch the interactive UI!

## 1️⃣ Check GPU

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("❌ No GPU! Go to Runtime → Change runtime type → GPU")

## 2️⃣ Clone Repository

⚠️ **Replace `YOUR_USERNAME` with your actual GitHub username!**

In [ ]:
# Clone your repository (update with your GitHub username)
!git clone https://github.com/beginner4a3/ui.git
%cd ui

print("✅ Repository cloned!")

## 3️⃣ Install Dependencies

In [ ]:
# Install all dependencies
!pip install -q gradio>=4.0.0
!pip install -q git+https://github.com/huggingface/parler-tts.git
!pip install -q transformers accelerate soundfile scipy

print("✅ Dependencies installed!")

## 4️⃣ Launch Interactive UI

This will:
1. Start the Gradio interface
2. Generate a **public URL** you can access
3. Let you control all audio quality settings!

In [ ]:
# Launch the Gradio app
!python app.py

---

## 📋 Alternative: Run Inline

If you prefer to run the UI directly in this notebook (without cloning):

In [ ]:
# Skip this if you already ran the cells above
# This is an alternative way to run the UI inline

import torch
import gradio as gr

# Configuration
SPEAKERS = ["-- Random Voice --", "Divya (Hindi)", "Rohit (Hindi)", "Maya (Hindi)", 
            "Karan (Hindi)", "Aditi (Tamil)", "Sunita (Tamil)", "Anjali (Telugu)"]
EMOTIONS = ["None", "Neutral", "Happy", "Sad", "Anger", "Fear", "Narration", "News"]

PITCH_MAP = {1: "low-pitched", 2: "slightly low-pitched", 3: "moderate pitch", 
             4: "slightly high-pitched", 5: "high-pitched"}
SPEED_MAP = {1: "slow pace", 2: "slightly slow pace", 3: "moderate pace",
             4: "slightly fast pace", 5: "fast pace"}
EXPR_MAP = {1: "monotone", 2: "slightly expressive", 3: "expressive and animated"}

# Model (global)
model = None
tokenizer = None
desc_tokenizer = None
device = None

def load_model():
    global model, tokenizer, desc_tokenizer, device
    from parler_tts import ParlerTTSForConditionalGeneration
    from transformers import AutoTokenizer
    
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
    
    model = ParlerTTSForConditionalGeneration.from_pretrained(
        "ai4bharat/indic-parler-tts", torch_dtype=dtype, attn_implementation="sdpa"
    ).to(device)
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
    desc_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)
    return f"✅ Model loaded on {device}"

def generate(text, speaker, gender, emotion, pitch, speed, expr, quality, noise, reverb):
    global model, tokenizer, desc_tokenizer, device
    if model is None:
        return None, "❌ Load model first!"
    
    # Build description
    if speaker != "-- Random Voice --":
        name = speaker.split(" (")[0]
        desc = f"{name}'s voice is {EXPR_MAP[expr]} with a {PITCH_MAP[pitch]} tone at a {SPEED_MAP[speed]}"
    else:
        desc = f"A {gender} speaker with a {PITCH_MAP[pitch]} voice delivers {EXPR_MAP[expr]} speech at a {SPEED_MAP[speed]}"
    
    if emotion != "None":
        desc += f" with a {emotion} tone"
    desc += f". The recording is of {quality}, with {noise} audio and a {reverb} environment."
    
    # Generate
    desc_ids = desc_tokenizer(desc, return_tensors="pt").to(device)
    text_ids = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        gen = model.generate(
            input_ids=desc_ids.input_ids, attention_mask=desc_ids.attention_mask,
            prompt_input_ids=text_ids.input_ids, prompt_attention_mask=text_ids.attention_mask
        )
    
    audio = gen.cpu().numpy().squeeze()
    return (model.config.sampling_rate, audio), f"✅ Done!\n\n📝 {desc}"

# Build UI
with gr.Blocks(theme=gr.themes.Soft(primary_hue="purple")) as app:
    gr.Markdown("# 🎤 Indic Parler TTS - Audio Quality Control")
    
    with gr.Row():
        load_btn = gr.Button("🚀 Load Model", variant="primary")
        status = gr.Textbox(label="Status", value="⏳ Click Load Model")
    load_btn.click(load_model, outputs=status)
    
    with gr.Row():
        with gr.Column():
            text = gr.Textbox(label="Text", value="Hello, welcome to Indic Parler TTS!", lines=3)
            speaker = gr.Dropdown(SPEAKERS, value="-- Random Voice --", label="Speaker")
            with gr.Row():
                gender = gr.Radio(["female", "male"], value="female", label="Gender")
                emotion = gr.Dropdown(EMOTIONS, value="None", label="Emotion")
            pitch = gr.Slider(1, 5, 3, step=1, label="Pitch")
            speed = gr.Slider(1, 5, 3, step=1, label="Speed")
            expr = gr.Slider(1, 3, 2, step=1, label="Expressivity")
            quality = gr.Radio(["very high quality", "high quality", "good quality"], 
                              value="very high quality", label="Quality")
            noise = gr.Radio(["very clear", "slightly noisy", "noisy"], 
                            value="very clear", label="Noise")
            reverb = gr.Radio(["close-sounding", "slightly distant", "distant-sounding"],
                             value="close-sounding", label="Reverb")
        
        with gr.Column():
            gen_btn = gr.Button("🎙️ Generate", variant="primary", size="lg")
            audio_out = gr.Audio(label="Output")
            status_out = gr.Textbox(label="Info", lines=4)
    
    gen_btn.click(generate, 
                  [text, speaker, gender, emotion, pitch, speed, expr, quality, noise, reverb],
                  [audio_out, status_out])

app.launch(share=True)

---

## 📖 Audio Quality Settings Reference

| Setting | Best for Clarity |
|---------|------------------|
| **Pitch** | Moderate (3) |
| **Speed** | Moderate (3) |
| **Expressivity** | Slightly Expressive (2) |
| **Quality** | Very High Quality |
| **Noise** | Very Clear |
| **Reverb** | Close-Sounding |

**Pro Tips:**
- Use named speakers (Divya, Karan) for consistent voice
- Add punctuation in text for natural pauses
- Emotions work best with Tamil, Bengali, Hindi